In [1]:
import pandas as pd
import numpy as np
import gc
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from scipy import stats
import pickle 
from pickle import load,dump
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier, log_evaluation
import warnings
import joblib

In [2]:
oof_xgb = pd.read_csv('../input/fork-of-amex-finalxgb/oof_transfer_learning.csv')
oof_xgbpy = pd.read_csv('../input/amex-fe-pyramid/oof_xgb_v1.csv') 
oof_automl = pd.read_csv('../input/amex-lightautoml/AutoML_oof.csv')
oof_automl43 = pd.read_csv('../input/amex-lightautoml43/AutoML_oof.csv')
oof_cat = pd.read_csv('../input/amex-catboost770/oof_cat_v1.csv')
oof_cat41 = pd.read_csv('../input/amex-catboost770-seed41/oof_cat_seed41.csv')
oof_lgbm = pd.read_csv('../input/lgbm770/oof_lgbm_v1(1).csv')
oof_lgbmquick = pd.read_csv('../input/amexlightgbmquickstart2seed/oof_lgbmquick.csv')
oof_transfer1 = pd.read_csv('../input/amextransferlearning/oof_transfer_learning.csv')
oof_transfer2 = pd.read_csv('../input/amextransferlearningresnet/oof_transfer_learning.csv')
oof_cnn = pd.read_csv('../input/amex-keras-1d-cnn/oof_CNN1D.csv')

In [3]:
oof_cat['oof_pred_cat'] = (oof_cat['oof_pred_cat'] + oof_cat41['oof_pred_cat'])/2

In [4]:
oof_automl['oof_pred'] = (oof_automl['oof_pred'] + oof_automl43['oof_pred'])/2

In [5]:
oof_xgb.rename(columns={'oof_pred':'test_preds_xgb'},inplace=True)
oof_xgbpy.rename(columns={'oof_pred':'test_preds_xgbpy'},inplace=True)
oof_cat.rename(columns={'oof_pred_cat':'test_preds_cat'},inplace=True)
oof_automl.rename(columns={'oof_pred':'test_preds_automl'},inplace=True)
oof_transfer1.rename(columns={'oof_pred':'test_preds_transfer1'},inplace=True)
oof_transfer2.rename(columns={'oof_pred':'test_preds_transfer2'},inplace=True)
oof_cnn.rename(columns={'oof_pred':'test_preds_cnn'},inplace=True)
oof_lgbm.rename(columns={'oof_pred_lgbm':'test_preds_lgbm'},inplace=True)
oof_lgbmquick.rename(columns={'oof_pred':'test_preds_lgbmquick'},inplace=True)

In [6]:
oof_df = pd.concat([oof_xgb,oof_automl.test_preds_automl,oof_lgbm.test_preds_lgbm,oof_xgbpy.test_preds_xgbpy,oof_cat.test_preds_cat,oof_lgbmquick.test_preds_lgbmquick,oof_cnn.test_preds_cnn,oof_transfer1.test_preds_transfer1,oof_transfer2.test_preds_transfer2],axis=1)

In [7]:
oof_df.head(5)

,target,test_preds_xgb,test_preds_automl,test_preds_lgbm,test_preds_xgbpy,test_preds_cat,test_preds_lgbmquick,test_preds_cnn,test_preds_transfer1,test_preds_transfer2
0,1,1.154194,0.782662,1.056450,1.028873,0.708636,1.310415,0.611922,0.760796,0.664672
1,0,-7.939376,0.000712,0.000387,-8.107467,0.000534,-7.767016,0.049239,0.000401,0.000582
2,0,-6.199133,0.001576,0.001538,-6.588468,0.000739,-6.753305,0.069256,0.001334,0.001924
3,0,-3.873612,0.019122,0.022494,-3.595696,0.027202,-3.918158,0.184326,0.021304,0.030478
4,1,1.994368,0.918872,2.121106,2.096829,0.921224,1.792675,0.626901,0.852713,0.859151


In [8]:
def amex_metric_mod(y_true, y_pred):

    labels     = np.transpose(np.array([y_true, y_pred]))
    labels     = labels[labels[:, 1].argsort()[::-1]]
    weights    = np.where(labels[:,0]==0, 20, 1)
    cut_vals   = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four   = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])

    gini = [0,0]
    for i in [1,0]:
        labels         = np.transpose(np.array([y_true, y_pred]))
        labels         = labels[labels[:, i].argsort()[::-1]]
        weight         = np.where(labels[:,0]==0, 20, 1)
        weight_random  = np.cumsum(weight / np.sum(weight))
        total_pos      = np.sum(labels[:, 0] *  weight)
        cum_pos_found  = np.cumsum(labels[:, 0] * weight)
        lorentz        = cum_pos_found / total_pos
        gini[i]        = np.sum((lorentz - weight_random) * weight)

    return 0.5 * (gini[1]/gini[0] + top_four)

In [9]:
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex',
            amex_metric(y_true, y_pred),
            True)

In [10]:
oofCols = [col for col in oof_df.columns if 'test_preds' in col]

In [11]:
FOLDS = 5
SEED = 42
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
ix=0
params = {'objective': 'CrossEntropy', 'colsample_bylevel': 0.5, 'bootstrap_type': 'Bernoulli', 
 'max_depth': 7, 'l2_leaf_reg': 37, 'random_strength': 0.9, 'subsample': 0.9}
oof_catstack42 = np.zeros((len(oof_df)))
for train_ind, val_ind in skf.split(oof_df, oof_df.target):
    
    print(f"******* Fold {ix} ******* ")
    tr_x, val_x = (
        oof_df[oofCols].iloc[train_ind].reset_index(drop=True),
        oof_df[oofCols].iloc[val_ind].reset_index(drop=True),
    )
    tr_y, val_y = (
        oof_df.target.iloc[train_ind].reset_index(drop=True),
        oof_df.target.iloc[val_ind].reset_index(drop=True),
    )

    clf = CatBoostClassifier(**params, iterations= 15500,  random_state=SEED ,allow_writing_files=False)
    clf.fit(tr_x, tr_y, eval_set=[(val_x, val_y)], verbose=100,early_stopping_rounds=200)
    clf.save_model(f'cat_stacked_fold{ix}_seed{SEED}')
    clf.load_model(f'./cat_stacked_fold{ix}_seed{SEED}')  
    preds = clf.predict_proba(val_x)[:,1]
    acc = amex_metric_mod(val_y.values, preds)
    print('Kaggle Metric =',acc,'\n')
    
    # SAVE OOF
    oof_catstack42[val_ind] = preds 
    
    del clf, tr_x, val_x, tr_y, val_y
    _ = gc.collect()
    
    ix = ix + 1
    
print('#'*25)
val_score = amex_metric_mod(oof_df.target, oof_catstack42)
print(f"Amex metric: {val_score}")

******* Fold 0 ******* 
0:	learn: 0.6435303	test: 0.6438022	best: 0.6438022 (0)	total: 132ms	remaining: 34m 7s
100:	learn: 0.2138984	test: 0.2141819	best: 0.2141819 (100)	total: 6.42s	remaining: 16m 19s
200:	learn: 0.2128113	test: 0.2133147	best: 0.2133147 (200)	total: 12.5s	remaining: 15m 53s
300:	learn: 0.2123918	test: 0.2132580	best: 0.2132558 (289)	total: 18.4s	remaining: 15m 30s
400:	learn: 0.2119818	test: 0.2132550	best: 0.2132465 (375)	total: 24.4s	remaining: 15m 18s
500:	learn: 0.2113660	test: 0.2132893	best: 0.2132465 (375)	total: 30.6s	remaining: 15m 16s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.2132465262
bestIteration = 375

Shrink model to first 376 iterations.
Kaggle Metric = 0.79688237874649 

******* Fold 1 ******* 
0:	learn: 0.6477317	test: 0.6476353	best: 0.6476353 (0)	total: 64.1ms	remaining: 16m 33s
100:	learn: 0.2138422	test: 0.2143378	best: 0.2143378 (100)	total: 6.61s	remaining: 16m 47s
200:	learn: 0.2127008	test: 0.2136572	best: 0.2136

In [12]:
oof_catstack = pd.DataFrame(oof_catstack42,columns=['oof_catstack42'])

In [13]:
oof_catstack.to_csv('oof_catstack.csv',index=False)